In [17]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

As explained in the report, hardcode latencies for critical points and establish a function to obtain the CPU latency between Core #0 and Core#p. Such function trivially interpolates the CPU latency assigning the same value found at the critical point until the next one is reached. 

In [18]:
latencies = {
  "1": 0.14,   # CCX
  "4": 0.35,   # CCD
  "8": 0.35,   # NUMA
  "32": 0.41,  # socket 
  "64": 0.65,  # node 
  "128": 1.82, # cluster
}
def get_latency(p):
  for k in latencies.keys():
    if int(k) >= p:
      return latencies[k]
  return latencies[list(latencies.keys())[-1]]
df = pd.DataFrame(data={
  "cpu": range(2, 256 + 2),
  "avg_latency": [get_latency(i) for i in range(256)]
}).dropna()

Derive the time spent for a chain of np processes by converting the 0 -> p latencies into p_i -> p_(i + 1) latencies. As explained in the report, this is needed due to the implementation of the chain algorithm, based on the exchanges between neighbouring processes.  

In [19]:
def get_chain_time(np):
  def get_latency(a, b):
    if a // 4 == b // 4:
      return latencies["1"]
    if a // 8 == b // 8:
      return latencies["4"]
    if a // 16 == b // 16:
      return latencies["8"]
    if a // 64 == b // 64:
      return latencies["32"]
    if a // 128 == b // 128:
      return latencies["64"]
    return latencies["128"]
  return sum([get_latency(i, i + 1) for i in range(np - 1)])

Load measured values for the Broadcast collective using the Chain algorithm and calculate its estimate based on an empirical formula, as explained in the report.

In [20]:
actual = pd.read_csv("data/20240222_204543_bcast2_msize2.txt")
df["actual_bcast"] = actual
df["estimate_bcast"] = df.apply(lambda e: 0.4 * get_chain_time(int(e["cpu"]) - 1), axis=1)

In [21]:
df

,cpu,avg_latency,actual_bcast,estimate_bcast
0,2,0.14,0.16,0.000
1,3,0.14,0.28,0.056
2,4,0.35,0.16,0.112
3,5,0.35,0.25,0.168
4,6,0.35,0.31,0.308
...,...,...,...,...
251,253,1.82,24.96,20.380
252,254,1.82,21.01,20.520
253,255,1.82,20.96,20.576
254,256,1.82,9.19,20.632


In [22]:
fig = px.scatter(df, x="cpu", y="actual_bcast")
fig.add_trace(go.Scatter(x=df["cpu"], y=df["estimate_bcast"], mode="lines", name="est_bcast"))